In [7]:
desk_jockeys = [{'name': 'Brandon',
                 'county': 'Lake',
                 'shift': 'AM'},
                {'name': 'David',
                 'county': 'Lake',
                 'shift': 'PM'},
                {'name': 'Brad',
                 'county': 'Marion',
                 'shift': 'AM'},
                {'name': 'Kody',
                 'county': 'Marion',
                 'shift': 'PM'},
                {'name': 'John',
                 'county': 'Hernando',
                 'shift': 'Either'},
                {'name': 'Rick',
                 'county': 'Hernando',
                 'shift': 'Either'},
                {'name': 'Ronan',
                 'county': 'Citrus',
                 'shift': 'AM'}]

def neighbors(i, lst, dist=1):
        llen = len(lst)
        if llen == 0:
                return([])
        elif i-dist < 0:
                return(lst[llen+i-dist:] + lst[0:i] + lst[i+1:i+1+dist])
        elif i+dist >= llen:
                return(lst[i-dist:i] + lst[i+1:llen] + lst[0:1+i+dist-llen])
        else:
                return(lst[i-dist:i] + lst[i+1:i+1+dist])

def deviation(lst):
        mean = sum(lst)/len(lst)
        devsum = 0
        for i in lst:
                devsum += (i-mean)**2

        return(devsum)

def eval_func(chromosome):
        score = 0.0
        for position, jockey in enumerate(chromosome):
                if jockey['county'] in [j['county'] for j in neighbors(position, chromosome)]:
                        # Avoid hitting one county sequentially
                        score += 0.5
                if jockey['name'] in [j['name'] for j in neighbors(position, chromosome, 2)]:
                        score += 0.4
                if jockey['shift'] == 'AM' and position % 2 != 0:
                        # Honor shift preference
                        score += 1.0
                elif jockey['shift'] == 'PM' and position % 2 == 0:
                        score += 1.0

        return(score)



In [8]:
from math import exp
from random import random, shuffle

def accept_probability(e1, e2, temperature):
        if e2 > e1:
                return(exp((e1-e2)/temperature))
        else:
                return(0.8)

def anneal(state, energy, acceptance, kmax=1000000):
        for k in range(1,kmax+1):
                temperature = kmax/k
                if k*10 % kmax == 0:
                        print(int(100*k/kmax), [j['name'] for j in state], energy(state), temperature)
                neighbor = state[:]
                shuffle(neighbor)
                if accept_probability(energy(state),
                                      energy(neighbor),
                                      temperature) >= random():
                        state = neighbor

        return(state, energy(state))


results = anneal(desk_jockeys,
                 eval_func,
                 accept_probability)

print(100, [j['name'] for j in results[0]], results[1])

100 ['Brad', 'Brandon', 'Rick', 'Ronan', 'John', 'David'] 2.0 1.0
100 ['Brad', 'Brandon', 'Rick', 'Ronan', 'John', 'David'] 2.0


90 ['David', 'Rick', 'Brad', 'John', 'Ronan', 'Brandon'] 3.0 1.1111111111111112


80 ['Ronan', 'John', 'Rick', 'David', 'Brad', 'Brandon'] 2.0 1.25


70 ['Brad', 'Ronan', 'Rick', 'Brandon', 'John', 'David'] 2.0 1.4285714285714286


60 ['Rick', 'David', 'John', 'Ronan', 'Brandon', 'Brad'] 2.0 1.6666666666666667


50 ['Rick', 'Ronan', 'David', 'Brad', 'Brandon', 'John'] 4.0 2.0


40 ['John', 'David', 'Brandon', 'Ronan', 'Rick', 'Brad'] 3.0 2.5


30 ['Brad', 'Ronan', 'Rick', 'John', 'Brandon', 'David'] 3.0 3.3333333333333335


20 ['Rick', 'David', 'John', 'Ronan', 'Brad', 'Brandon'] 2.0 5.0


10 ['Rick', 'Brad', 'David', 'Ronan', 'John', 'Brandon'] 4.0 10.0
